In [49]:
import pandas as pd
import numpy as np
import os
import keras
from datetime import timedelta, datetime

In [208]:
#os.chdir("src") #used to reset to original working directory

FileNotFoundError: [Errno 2] No such file or directory: 'src'

In [3]:
os.chdir("../") #Used to go out into the cmpe-351-group-1 folder to access the data without changing directories multiple times

In [50]:
def hour_rounder(t):
    # Rounds to nearest hour by adding a timedelta hour if minute >= 30
    return (t.replace(second=0, microsecond=0, minute=0, hour=t.hour)
            +timedelta(hours=t.minute//30))

In [51]:
##Dates is an array of tuples consisting of month and day as numbers
def get_dataset(crypto,start_month,start_day,end_month,end_day):
    twitter_df = pd.read_csv(f'data/processed/twitter/{crypto}_sentiment.csv')
    try:
        bit_prices = pd.read_csv(f'data/raw/crypto/{crypto_dict[crypto]}_2022_{start_month}_{start_day}-2022_{end_month}_{end_day}_minute.csv')
    except:
        bit_prices = pd.read_csv(f'data/raw/crypto/{crypto_dict[crypto]}_2022_{start_month}_{start_day}-2022_{end_month}_{end_day}.csv')
    bit_prices["time"] = pd.to_datetime(bit_prices['time'],unit='s')
    price_dict = bit_prices.set_index("time")["open"].to_dict()
    twitter_df["created_at"] = twitter_df["created_at"].apply(lambda x: pd.to_datetime(x).tz_localize(None))
    twitter_df["created_at"] = twitter_df["created_at"].apply(lambda x: hour_rounder(x))
    twitter_df["future_date"] = twitter_df["created_at"].apply(lambda x: x+timedelta(hours=23))

    twitter_df = twitter_df[twitter_df["future_date"].isin(bit_prices["time"])] ##Filter out dates for which we don't have price data

    #print(len(twitter_df))

    twitter_df = twitter_df[twitter_df["created_at"].isin(bit_prices["time"])] ##Filter out dates for which we don't have price data


    twitter_df["price"] = twitter_df["created_at"].apply(lambda x: price_dict[x])
    for i in range(1,24):
        twitter_df[f"price_{i}hours"] = twitter_df["created_at"].apply(lambda x: price_dict[x+timedelta(hours=i)])
    X = twitter_df[["retweet_count","vader_sentiment_compound","price"]]
    y = twitter_df[[f"price_{i}hours" for i in range(1,24)]]
    return twitter_df,X,y


In [52]:
df_avax,X_avax,y_avax = get_dataset("avalanche","03","05","03","11")
df_avax2, X_avax2, y_avax2 = get_dataset("avalanche","03","28","04","04")
df_avax3, X_avax3, y_avax3 = get_dataset("avalanche","03","11","03","12")
df_avax4, X_avax4, y_avax4 = get_dataset("avalanche","04","04","04","05")

df_avax = df_avax.append(df_avax2)
df_avax = df_avax.append(df_avax3)
df_avax = df_avax.append(df_avax4)

X_avax = X_avax.append(X_avax2)
X_avax = X_avax.append(X_avax3)
X_avax = X_avax.append(X_avax4)

y_avax = y_avax.append(y_avax2)
y_avax = y_avax.append(y_avax3)
y_avax = y_avax.append(y_avax4)

df_avax_test, _,_ = get_dataset("avalanche","04","05","04","07")


In [53]:
df_btc,X_btc,y_btc = get_dataset("bitcoin","03","05","03","11")
df_btc2, X_btc2, y_btc2 = get_dataset("bitcoin","03","28","04","04")
df_btc3, X_btc3, y_btc3 = get_dataset("bitcoin","03","11","03","12")
df_btc4, X_btc4, y_btc4 = get_dataset("bitcoin","04","04","04","05")

df_btc = df_btc.append(df_btc2)
df_btc = df_btc.append(df_btc3)
df_btc = df_btc.append(df_btc4)

X_btc = X_btc.append(X_btc2)
X_btc = X_btc.append(X_btc3)
X_btc = X_btc.append(X_btc4)

y_btc = y_btc.append(y_btc2)
y_btc = y_btc.append(y_btc3)
y_btc = y_btc.append(y_btc4)
df_btc_test, _,_ = get_dataset("bitcoin","04","05","04","07")


In [54]:
df_eth,X_eth,y_eth = get_dataset("ethereum","03","05","03","11")
df_eth2, X_eth2, y_eth2 = get_dataset("ethereum","03","28","04","04")
df_eth3,X_eth3,y_eth3 = get_dataset("ethereum","03","11","03","12")
df_eth4, X_eth4, y_eth4 = get_dataset("ethereum","04","04","04","05")

df_eth = df_eth.append(df_eth2)
df_eth = df_eth.append(df_eth3)
df_eth = df_eth.append(df_eth4)



X_eth = X_eth.append(X_eth2)
X_eth = X_eth.append(X_eth3)
X_eth = X_eth.append(X_eth4)

y_eth = y_eth.append(y_eth2)
y_eth = y_eth.append(y_eth3)
y_eth = y_eth.append(y_eth4)

df_eth_test, _,_ = get_dataset("ethereum","04","05","04","07")

In [55]:
df_sol,X_sol,y_sol = get_dataset("solana","03","05","03","11")
df_sol2, X_sol2, y_sol2 = get_dataset("solana","03","28","04","04")
df_sol3,X_sol3,y_sol3 = get_dataset("solana","03","11","03","12")
df_sol4, X_sol4, y_sol4 = get_dataset("solana","04","04","04","05")

df_sol = df_sol.append(df_sol2)
df_sol = df_sol.append(df_sol3)
df_sol = df_sol.append(df_sol4)


X_sol = X_sol.append(X_sol2)
X_sol = X_sol.append(X_sol3)
X_sol = X_sol.append(X_sol4)

y_sol = y_sol.append(y_sol2)
y_sol = y_sol.append(y_sol3)
y_sol = y_sol.append(y_sol4)

df_sol_test, _,_ = get_dataset("solana","04","05","04","07")

In [56]:
df_doge,X_doge,y_doge = get_dataset("doge","03","05","03","11")
df_doge2, X_doge2, y_doge2 = get_dataset("doge","03","28","04","04")
df_doge3,X_doge3,y_doge3 = get_dataset("doge","03","11","03","12")
df_doge4, X_doge4, y_doge4 = get_dataset("doge","04","04","04","05")

df_doge = df_doge.append(df_doge2)
df_doge = df_doge.append(df_doge3)
df_doge = df_doge.append(df_doge4)


X_doge = X_doge.append(X_doge2)
X_doge = X_doge.append(X_doge3)
X_doge = X_doge.append(X_doge4)

y_doge = y_doge.append(y_doge2)
y_doge = y_doge.append(y_doge3)
y_doge = y_doge.append(y_doge4)

df_doge_test, _,_ = get_dataset("solana","04","05","04","07")

In [57]:
def estimate_price(model,date,df):
    date_23_hours = date-timedelta(hours=23)
    date_1_hours = date-timedelta(hours=1)
    df_useful = df[(df["created_at"]<=date_1_hours) & (df["created_at"]>=date_23_hours)]
    if len(df_useful)==0:
        return ""
    else:
        df_useful["idx"] = df_useful["created_at"].apply(lambda x: round((date-x).seconds/60/60))
        X = df_useful[["retweet_count","vader_sentiment_compound","price"]]
        y_hat = model.predict_on_batch(X)
        idx = df_useful["idx"].values
        vals = []
        for i in range(len(y_hat)):
            vals.append(y_hat[i][idx[i]-1])
        return np.mean(vals)

In [58]:
def make_model():
    model = keras.Sequential()
    model.add(keras.layers.LSTM(8,input_shape= (3,1)))
    model.add(keras.layers.Dense(23))
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])
    return model

In [59]:
def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)*24):
        yield start_date + timedelta(hours=n)

In [60]:
def make_prediction(start_date, end_date, model,df):
    pred = {}
    for date in daterange(datetime.strptime(start_date,"%d/%m/%Y"),datetime.strptime(end_date,"%d/%m/%Y")):
        val = estimate_price(model,date,df)
        if val!='':
            pred[date.strftime("%d/%m/%Y %H:%M:%S")] = val
    return pred

In [61]:
bit_model = make_model()
bit_model.fit(X_btc,y_btc,epochs=1000,batch_size=10)

Epoch 1/1000
163/163 [==============================] - 3s 3ms/step - loss: 2023858688.0000 - mean_squared_error: 2023858688.0000
Epoch 2/1000
163/163 [==============================] - 0s 3ms/step - loss: 2023833728.0000 - mean_squared_error: 2023833728.0000
Epoch 3/1000
163/163 [==============================] - 1s 3ms/step - loss: 2023808000.0000 - mean_squared_error: 2023808000.0000
Epoch 4/1000
163/163 [==============================] - 1s 4ms/step - loss: 2023782272.0000 - mean_squared_error: 2023782272.0000
Epoch 5/1000
163/163 [==============================] - 1s 4ms/step - loss: 2023756032.0000 - mean_squared_error: 2023756032.0000
Epoch 6/1000
163/163 [==============================] - 0s 3ms/step - loss: 2023730816.0000 - mean_squared_error: 2023730816.0000
Epoch 7/1000
163/163 [==============================] - 0s 3ms/step - loss: 2023704192.0000 - mean_squared_error: 2023704192.0000
Epoch 8/1000
163/163 [==============================] - 0s 3ms/step - loss: 2023679232.000

In [62]:
eth_model = make_model()
eth_model.fit(X_eth,y_eth,epochs=1000,batch_size=10)

Epoch 1/1000
141/141 [==============================] - 2s 2ms/step - loss: 10642486.0000 - mean_squared_error: 10642487.0000
Epoch 2/1000
141/141 [==============================] - 0s 2ms/step - loss: 10634385.0000 - mean_squared_error: 10634385.0000
Epoch 3/1000
141/141 [==============================] - 0s 2ms/step - loss: 10626355.0000 - mean_squared_error: 10626355.0000
Epoch 4/1000
141/141 [==============================] - 0s 2ms/step - loss: 10617725.0000 - mean_squared_error: 10617725.0000
Epoch 5/1000
141/141 [==============================] - 0s 2ms/step - loss: 10608397.0000 - mean_squared_error: 10608397.0000
Epoch 6/1000
141/141 [==============================] - 0s 2ms/step - loss: 10599798.0000 - mean_squared_error: 10599798.0000
Epoch 7/1000
141/141 [==============================] - 0s 2ms/step - loss: 10591572.0000 - mean_squared_error: 10591572.0000
Epoch 8/1000
141/141 [==============================] - 0s 2ms/step - loss: 10583543.0000 - mean_squared_error: 105835

In [63]:
doge_model = make_model()
doge_model.fit(X_doge,y_doge,epochs=1000,batch_size=10)

Epoch 1/1000
50/50 [==============================] - 2s 2ms/step - loss: 0.0205 - mean_squared_error: 0.0205
Epoch 2/1000
50/50 [==============================] - 0s 2ms/step - loss: 0.0042 - mean_squared_error: 0.0042
Epoch 3/1000
50/50 [==============================] - 0s 2ms/step - loss: 6.8440e-04 - mean_squared_error: 6.8440e-04
Epoch 4/1000
50/50 [==============================] - 0s 2ms/step - loss: 2.1465e-04 - mean_squared_error: 2.1465e-04
Epoch 5/1000
50/50 [==============================] - 0s 2ms/step - loss: 1.5293e-04 - mean_squared_error: 1.5293e-04
Epoch 6/1000
50/50 [==============================] - 0s 2ms/step - loss: 1.4533e-04 - mean_squared_error: 1.4533e-04
Epoch 7/1000
50/50 [==============================] - 0s 2ms/step - loss: 1.4222e-04 - mean_squared_error: 1.4222e-04
Epoch 8/1000
50/50 [==============================] - 0s 2ms/step - loss: 1.4182e-04 - mean_squared_error: 1.4182e-04
Epoch 9/1000
50/50 [==============================] - 0s 2ms/step - loss

In [64]:
sol_model = make_model()
sol_model.fit(X_sol,y_sol,epochs=1000,batch_size=10)

Epoch 1/1000
32/32 [==============================] - 2s 2ms/step - loss: 14352.8877 - mean_squared_error: 14352.8877
Epoch 2/1000
32/32 [==============================] - 0s 2ms/step - loss: 14331.5547 - mean_squared_error: 14331.5547
Epoch 3/1000
32/32 [==============================] - 0s 2ms/step - loss: 14287.5518 - mean_squared_error: 14287.5518
Epoch 4/1000
32/32 [==============================] - 0s 2ms/step - loss: 14244.9971 - mean_squared_error: 14244.9971
Epoch 5/1000
32/32 [==============================] - 0s 2ms/step - loss: 14164.7256 - mean_squared_error: 14164.7256
Epoch 6/1000
32/32 [==============================] - 0s 2ms/step - loss: 14050.0732 - mean_squared_error: 14050.0732
Epoch 7/1000
32/32 [==============================] - 0s 2ms/step - loss: 13970.9648 - mean_squared_error: 13970.9648
Epoch 8/1000
32/32 [==============================] - 0s 2ms/step - loss: 13903.9775 - mean_squared_error: 13903.9775
Epoch 9/1000
32/32 [==============================] - 0s

In [65]:
avax_model = make_model()
avax_model.fit(X_avax,y_avax,epochs=1000,batch_size=10)

Epoch 1/1000
5/5 [==============================] - 1s 3ms/step - loss: 6696.4658 - mean_squared_error: 6696.4658
Epoch 2/1000
5/5 [==============================] - 0s 2ms/step - loss: 6694.0488 - mean_squared_error: 6694.0488
Epoch 3/1000
5/5 [==============================] - 0s 2ms/step - loss: 6691.5977 - mean_squared_error: 6691.5977
Epoch 4/1000
5/5 [==============================] - 0s 2ms/step - loss: 6689.1582 - mean_squared_error: 6689.1582
Epoch 5/1000
5/5 [==============================] - 0s 2ms/step - loss: 6686.7031 - mean_squared_error: 6686.7031
Epoch 6/1000
5/5 [==============================] - 0s 2ms/step - loss: 6684.2148 - mean_squared_error: 6684.2148
Epoch 7/1000
5/5 [==============================] - 0s 2ms/step - loss: 6681.7383 - mean_squared_error: 6681.7383
Epoch 8/1000
5/5 [==============================] - 0s 2ms/step - loss: 6679.2793 - mean_squared_error: 6679.2793
Epoch 9/1000
5/5 [==============================] - 0s 2ms/step - loss: 6676.8208 - mean

In [66]:
make_prediction("05/04/2022","07/04/2022",bit_model,df_btc_test)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


{'05/04/2022 12:00:00': 287.37637,
 '05/04/2022 13:00:00': 287.3351,
 '05/04/2022 14:00:00': 287.2959,
 '05/04/2022 15:00:00': 287.1793,
 '05/04/2022 16:00:00': 287.10626,
 '05/04/2022 17:00:00': 287.1449,
 '05/04/2022 18:00:00': 287.20706,
 '05/04/2022 19:00:00': 287.11743,
 '05/04/2022 20:00:00': 286.96283,
 '05/04/2022 21:00:00': 287.0419,
 '05/04/2022 22:00:00': 287.16187,
 '05/04/2022 23:00:00': 287.1006,
 '06/04/2022 00:00:00': 287.121,
 '06/04/2022 01:00:00': 287.04132,
 '06/04/2022 02:00:00': 287.14868,
 '06/04/2022 03:00:00': 287.16055,
 '06/04/2022 04:00:00': 287.0738,
 '06/04/2022 05:00:00': 287.18027,
 '06/04/2022 06:00:00': 287.0395,
 '06/04/2022 07:00:00': 287.066,
 '06/04/2022 08:00:00': 287.14603,
 '06/04/2022 09:00:00': 287.03094,
 '06/04/2022 10:00:00': 287.01532,
 '06/04/2022 11:00:00': 287.01608,
 '06/04/2022 12:00:00': 286.9485,
 '06/04/2022 13:00:00': 287.1585,
 '06/04/2022 14:00:00': 286.90564,
 '06/04/2022 15:00:00': 287.02008,
 '06/04/2022 16:00:00': 286.98367,